In [11]:
%uv pip install transformers safetensors huggingface-hub accelerate sentencepiece

Note: you may need to restart the kernel to use updated packages.


Using Python 3.11.5 environment at: D:\python\SparkMind
Resolved 27 packages in 1.46s
Prepared 2 packages in 1.51s
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 2 packages in 103ms
 + accelerate==1.10.1
 + sentencepiece==0.2.1


In [1]:
# ...existing code...
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch, shutil, os

model_name = "tabularisai/multilingual-sentiment-analysis"
cache_dir = r"D:\Models\Sentiment"

def load_model_and_tokenizer():
    try:
        tok = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)
        mdl = AutoModelForSequenceClassification.from_pretrained(
            model_name, cache_dir=cache_dir, torch_dtype=torch.float32
        )
        return tok, mdl
    except OSError:
        # Remove possibly corrupted snapshot and force re-download
        snapshot_root = os.path.join(cache_dir, "models--" + model_name.replace("/", "--"))
        if os.path.isdir(snapshot_root):
            shutil.rmtree(snapshot_root, ignore_errors=True)
        tok = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir, force_download=True)
        mdl = AutoModelForSequenceClassification.from_pretrained(
            model_name, cache_dir=cache_dir, force_download=True, torch_dtype=torch.float32
        )
        return tok, mdl

tokenizer, model = load_model_and_tokenizer()
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

d:\ForeKnow\backend\SparkMind\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)

In [ ]:
text = "I… I woke up late today. Um… then I went to the kitchen..."
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)
pred = torch.softmax(outputs.logits, dim=1)
labels = ["very negative", "negative", "neutral", "positive", "very positive"]

print(labels[torch.argmax(pred)], pred.tolist())

positive [[0.0428469218313694, 0.16647836565971375, 0.1968902051448822, 0.5565119981765747, 0.037272531539201736]]


: 

In [28]:
label_wts : [float] =[0.2,0.4,0.6,0.8,1.0]
weighted_sum: float = sum(p * w for p, w in zip(pred[0], label_wts))*100
print(f"Weighted sentiment score: {weighted_sum:.3f}")

Weighted sentiment score: 67.578


C:\Users\karee\AppData\Local\Temp\ipykernel_16604\1380080934.py:2: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  weighted_sum: float = sum(p * w for p, w in zip(pred[0], label_wts))*100
